In [3]:
from scholarly import scholarly

query = scholarly.search_author('Rafael Martinez Fonseca')
author = next(query)
print(author)


{'container_type': 'Author', 'filled': [], 'source': <AuthorSource.SEARCH_AUTHOR_SNIPPETS: 'SEARCH_AUTHOR_SNIPPETS'>, 'scholar_id': 'WCkRu_kAAAAJ', 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=WCkRu_kAAAAJ', 'name': 'Rafael Martínez Fonseca', 'affiliation': 'PhD student in mathematics', 'email_domain': '@mail.uniatlantico.edu.co', 'interests': ['Biomatemáticas'], 'citedby': 2}


In [4]:
import csv
from scholarly import scholarly


In [10]:

# Palabras clave de la búsqueda
query = '("Topological data analysis" OR "landscape persistence diagrams") AND ("Neuroimage" OR "Magnetic Resonance Image") AND ("confidence interval" OR "Estimation")'

# Realizar la búsqueda
search_results = scholarly.search_pubs(query)

# Crear y abrir un archivo CSV
filename = "scholarly_search_results.csv"
with open(filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # Escribir encabezados
    writer.writerow(["Título", "Autores", "Resumen", "Año", "URL"])

    # Recorrer resultados y escribir en el archivo
    for i, result in enumerate(search_results):
        if i >= 500:  # Cambia este número para controlar el número de artículos a guardar
            break
        try:
            # Extraer datos relevantes
            title = result.get("bib", {}).get("title", "Sin título")
            authors = ", ".join(result.get("bib", {}).get("author", []))
            abstract = result.get("bib", {}).get("abstract", "No disponible")
            year = result.get("bib", {}).get("pub_year", "No especificado")
            url = result.get("eprint_url", "No disponible")

            # Escribir en el CSV
            writer.writerow([title, authors, abstract, year, url])
        except Exception as e:
            print(f"Error al procesar un artículo: {e}")

print(f"Resultados guardados en el archivo {filename}")


MaxTriesExceededException: Cannot Fetch from Google Scholar.

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Función para construir la URL de búsqueda
def build_springer_url(query, page=1):
    base_url = "https://link.springer.com/search"
    return f"{base_url}?query={query.replace(' ', '+')}&page={page}"

# Encabezados para simular un navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Consulta de búsqueda
query = "Topological data analysis"
results = []

# Iterar sobre las páginas de resultados
for page in range(1, 4):  # Cambia 4 al número de páginas que quieres procesar
    url = build_springer_url(query, page)
    print(f"Scraping: {url}")
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Encontrar todos los artículos en la página
        articles = soup.select(".result-item-content")
        for article in articles:
            # Título del artículo
            title_element = article.select_one(".title")
            title = title_element.text.strip() if title_element else "No title available"
            
            # Enlace al artículo
            link = f"https://link.springer.com{title_element['href']}" if title_element else None
            
            # Resumen (si está disponible)
            snippet_element = article.select_one(".snippet")
            snippet = snippet_element.text.strip() if snippet_element else "No snippet available"
            
            # Guardar el resultado
            results.append({"Title": title, "Link": link, "Snippet": snippet})
        
        time.sleep(2)  # Pausa entre solicitudes para evitar ser bloqueado
    else:
        print(f"Error al acceder a la página: {response.status_code}")
        break

# Guardar resultados en un archivo CSV
df = pd.DataFrame(results)
df.to_csv("springer_results.csv", index=False)
print("Resultados guardados en springer_results.csv")


Scraping: https://link.springer.com/search?query=Topological+data+analysis&page=1
Scraping: https://link.springer.com/search?query=Topological+data+analysis&page=2
Scraping: https://link.springer.com/search?query=Topological+data+analysis&page=3
Resultados guardados en springer_results.csv


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Función para construir la URL de búsqueda
def build_sciencedirect_url(query, page=0):
    base_url = "https://www.sciencedirect.com/search"
    return f"{base_url}?qs={query.replace(' ', '+')}&offset={page * 25}"

# Encabezados para simular un navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Consulta de búsqueda
query = "Topological data analysis"
results = []

# Iterar sobre las páginas de resultados
for page in range(3):  # Cambia 3 al número de páginas que quieres procesar
    url = build_sciencedirect_url(query, page)
    print(f"Scraping: {url}")
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Encontrar todos los artículos en la página
        articles = soup.select(".result-item-content")
        for article in articles:
            # Título del artículo
            title_element = article.select_one(".result-list-title-link")
            title = title_element.text.strip() if title_element else "No title available"
            
            # Enlace al artículo
            link = f"https://www.sciencedirect.com{title_element['href']}" if title_element else None
            
            # Resumen (si está disponible)
            snippet_element = article.select_one(".result-list-snippet")
            snippet = snippet_element.text.strip() if snippet_element else "No snippet available"
            
            # Autores (si están disponibles)
            authors_element = article.select_one(".Authors")
            authors = authors_element.text.strip() if authors_element else "No authors available"
            
            # Guardar el resultado
            results.append({
                "Title": title,
                "Link": link,
                "Snippet": snippet,
                "Authors": authors
            })
        
        time.sleep(2)  # Pausa entre solicitudes para evitar ser bloqueado
    else:
        print(f"Error al acceder a la página: {response.status_code}")
        break

# Guardar resultados en un archivo CSV
df = pd.DataFrame(results)
df.to_csv("sciencedirect_results.csv", index=False)
print("Resultados guardados en sciencedirect_results.csv")



Scraping: https://www.sciencedirect.com/search?qs=Topological+data+analysis&offset=0
Error al acceder a la página: 403
Resultados guardados en sciencedirect_results.csv


In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Función para construir la URL de búsqueda
def build_pubmed_url(query, page=1):
    base_url = "https://pubmed.ncbi.nlm.nih.gov/"
    return f"{base_url}?term={query.replace(' ', '+')}&page={page}"

# Encabezados para simular un navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Consulta de búsqueda
query = "Topological data analysis"
results = []

# Iterar sobre las páginas de resultados
for page in range(1, 4):  # Cambia 4 al número de páginas que quieras procesar
    url = build_pubmed_url(query, page)
    print(f"Scraping: {url}")
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Encontrar todos los artículos en la página
        articles = soup.select(".docsum-content")
        for article in articles:
            # Título del artículo
            title_element = article.select_one(".docsum-title")
            title = title_element.text.strip() if title_element else "No title available"
            
            # Enlace al artículo
            link = f"https://pubmed.ncbi.nlm.nih.gov{title_element['href']}" if title_element else None
            
            # Autores (si están disponibles)
            authors_element = article.select_one(".docsum-authors.full-authors")
            authors = authors_element.text.strip() if authors_element else "No authors available"
            
            # Resumen (no está directamente en la lista de resultados, requiere visitar el enlace)
            snippet = "Abstract not available in search results"
            
            # Guardar el resultado
            results.append({
                "Title": title,
                "Link": link,
                "Authors": authors,
                "Snippet": snippet
            })
        
        time.sleep(2)  # Pausa entre solicitudes para evitar ser bloqueado
    else:
        print(f"Error al acceder a la página: {response.status_code}")
        break

# Guardar resultados en un archivo CSV
df = pd.DataFrame(results)
df.to_csv("pubmed_results.csv", index=False)
print("Resultados guardados en pubmed_results.csv")


Scraping: https://pubmed.ncbi.nlm.nih.gov/?term=Topological+data+analysis&page=1
Scraping: https://pubmed.ncbi.nlm.nih.gov/?term=Topological+data+analysis&page=2
Scraping: https://pubmed.ncbi.nlm.nih.gov/?term=Topological+data+analysis&page=3
Resultados guardados en pubmed_results.csv


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Función para construir la URL de búsqueda en PubMed
def build_pubmed_url(query, page=1):
    base_url = "https://pubmed.ncbi.nlm.nih.gov/"
    return f"{base_url}?term={query.replace(' ', '+')}&page={page}"

# Encabezados para simular un navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Clave ajustada para PubMed
query = '(("Topological data analysis"[Title/Abstract] OR "landscape persistence diagrams"[Title/Abstract]) AND ("Neuroimage"[Title/Abstract] OR "Magnetic Resonance Image"[Title/Abstract]) AND ("confidence interval"[Title/Abstract] OR "Estimation"[Title/Abstract]))'
results = []

# Iterar sobre las páginas de resultados
for page in range(1, 4):  # Cambia 4 al número de páginas que quieras procesar
    url = build_pubmed_url(query, page)
    print(f"Scraping: {url}")
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Encontrar todos los artículos en la página
        articles = soup.select(".docsum-content")
        for article in articles:
            # Título del artículo
            title_element = article.select_one(".docsum-title")
            title = title_element.text.strip() if title_element else "No title available"
            
            # Enlace al artículo
            link = f"https://pubmed.ncbi.nlm.nih.gov{title_element['href']}" if title_element else None
            
            # Autores (si están disponibles)
            authors_element = article.select_one(".docsum-authors.full-authors")
            authors = authors_element.text.strip() if authors_element else "No authors available"
            
            # Resumen (no está directamente en la lista de resultados, requiere visitar el enlace)
            snippet = "Abstract not available in search results"
            
            # Guardar el resultado
            results.append({
                "Title": title,
                "Link": link,
                "Authors": authors,
                "Snippet": snippet
            })
        
        time.sleep(2)  # Pausa entre solicitudes para evitar ser bloqueado
    else:
        print(f"Error al acceder a la página: {response.status_code}")
        break

# Guardar resultados en un archivo CSV
df = pd.DataFrame(results)
df.to_csv("pubmed_results2.csv", index=False)
print("Resultados guardados en pubmed_results2.csv")


Scraping: https://pubmed.ncbi.nlm.nih.gov/?term=(("Topological+data+analysis"[Title/Abstract]+OR+"landscape+persistence+diagrams"[Title/Abstract])+AND+("Neuroimage"[Title/Abstract]+OR+"Magnetic+Resonance+Image"[Title/Abstract])+AND+("confidence+interval"[Title/Abstract]+OR+"Estimation"[Title/Abstract]))&page=1
Scraping: https://pubmed.ncbi.nlm.nih.gov/?term=(("Topological+data+analysis"[Title/Abstract]+OR+"landscape+persistence+diagrams"[Title/Abstract])+AND+("Neuroimage"[Title/Abstract]+OR+"Magnetic+Resonance+Image"[Title/Abstract])+AND+("confidence+interval"[Title/Abstract]+OR+"Estimation"[Title/Abstract]))&page=2
Scraping: https://pubmed.ncbi.nlm.nih.gov/?term=(("Topological+data+analysis"[Title/Abstract]+OR+"landscape+persistence+diagrams"[Title/Abstract])+AND+("Neuroimage"[Title/Abstract]+OR+"Magnetic+Resonance+Image"[Title/Abstract])+AND+("confidence+interval"[Title/Abstract]+OR+"Estimation"[Title/Abstract]))&page=3
Resultados guardados en pubmed_results2.csv
